In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import open3d as o3d
from datetime import datetime

from utils import *
from bpy_utils import get_manus_data, get_fbx_creation_time

import sys
manopth_dir = 'C:/Users/lucas/Desktop/UPC/MIT/manopth/'
sys.path.insert(1, manopth_dir)

from manus.utils import Pressure

In [3]:
recording_dir = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/intrinsics_extrinsics/data/20210722/round_2_fixed_wooden_cube/"
recording_dir = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/intrinsics_extrinsics/data/20210722/round_3_fixed_bottle_pose_1/"


for name in os.listdir(recording_dir):
    name_pth = os.path.join(recording_dir, name)
    
    if os.path.isdir(name_pth):
        print("recording:", name)
        cameras_dir = name_pth
        
    elif ".hdf5" in name:
        print("pressure_file:", name)
        perssure_pth = name_pth
        
    elif ".fbx" in name:
        print("manus_file:", name)
        manus_pth = name_pth
        
#cameras_dir = os.path.join(recording_dir, "20210722_152500")
#perssure_pth = os.path.join(recording_dir, ".hdf5")
#manus_pth = os.path.join(recording_dir, ".fbx")

pressure_file: 07222021_3.hdf5
recording: 20210722_152847
manus_file: recording_2021-07-22_14-30-05.fbx


# Load data

### Camera data

In [4]:
cameras = ["020122061233", "821312060044", "020122061651", "821312062243"]

intrinsics = Intrinsics(cameras_dir)
extrinsics = Extrinsics(cameras_dir)
rgbds = RGBD(cameras_dir)
stitched_pcd = Stitching_pcds(intrinsics, extrinsics, rgbds, cameras)
apriltags = AprilTags(cameras_dir, intrinsics, extrinsics, cameras=cameras)
stitched_pcd = Stitching_pcds(intrinsics, extrinsics, rgbds, cameras)

### Pressure data

In [5]:
pressure = Pressure(perssure_pth, lim_low=3, lim_high=50)

### Manus data

Takes around two minutes to load the manus data and compute the MANO joints and verts

In [6]:
%%capture

hand_verts, hand_joints, hand_faces = get_manus_data(manus_pth, manopth_dir)

# Synchronize

#### Get timestamps

In [7]:
# Camera timestamps
ts_camera = get_camera_timestamps(cameras_dir)

# Pressure timestamps
ts_pressure = pressure.ts_touch

# MANUS timestamp
pyfbx_i42_pth = "C:/Users/lucas/Desktop/UPC/MIT/pyfbx_i42-master/"
ts_manus = get_fbx_creation_time(manus_pth, pyfbx_i42_pth)

In [8]:
datetime.fromtimestamp(ts_camera[-1]) - datetime.fromtimestamp(ts_camera[0])

datetime.timedelta(seconds=59, microseconds=627297)

In [9]:
datetime.fromtimestamp(ts_pressure[-1]) - datetime.fromtimestamp(ts_pressure[0])

datetime.timedelta(seconds=74, microseconds=100026)

#### Find offsets


In [10]:
# Index of first pressure peak 
idx_first_p = pressure.get_peaks()[0]
# Timestamp of first pressure peak
ts_first_p = pressure.ts_touch[idx_first_p]

# Frame of first pressure in camera
frame_first_c = 135
# Timestamp of first pressure in camera
ts_first_c = ts_camera[frame_first_c]

# Offset between pressure and camera
pc_offset = ts_first_p - ts_first_c

In [11]:
class Synchronizer:
    def __init__(self, ts_pressure, ts_camera, pc_offset=0):
        self.ts_pressure = ts_pressure - pc_offset
        self.ts_camera = ts_camera 
        
    def pressure_to_camera(self, frame_p):
        ts_p = self.ts_pressure[frame_p]
        frame_c = (np.abs(self.ts_camera - ts_p)).argmin()
        return frame_c
    
    def camera_to_pressure(self, frame_c):
        ts_c = self.ts_camera[frame_c]
        frame_p = (np.abs(self.ts_pressure - ts_c)).argmin()
        return frame_p 

In [12]:
synchronize = Synchronizer(ts_pressure, ts_camera, pc_offset)

In [13]:
datetime.fromtimestamp(ts_camera[0])

datetime.datetime(2021, 7, 22, 16, 28, 47, 965806)

# Save tactile img

In [ ]:
tactile_dir = os.path.join(apriltags.cameras_dir, "imgs", "pressure_img")

if not os.path.exists(tactile_dir):
    os.makedirs(tactile_dir)

for frame_c in range(len(ts_camera)):
    frame_p = synchronize.camera_to_pressure(frame_c)
    
    fig = pressure.get_figure(frame_p, show=False)
    img_pth = os.path.join(tactile_dir, f"img_{frame_c}")
    fig.savefig(img_pth)

# Merge camera + pressure + manus

#### Estimate object position

In [15]:
%%capture

pcd = stitched_pcd["000000"]

obj_name = "005_tomato_soup_can"
obj_name = "003_cracker_box"
obj_name = "036_wood_block"
obj_name = "006_mustard_bottle"

obj_pth = f"C:/Users/lucas/Desktop/UPC/MIT/tactile2object/estimate_object_pose/ycb_converted/{obj_name}.ply"

transformation, obj_mesh_t = find_object_transformation(pcd, obj_pth)

Manual ICP
Visualization of two point clouds before manual alignment

1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D INFO] Picked point #6687 (-0.013, -0.023, 0.19) to add in queue.
[Open3D INFO] Picked point #4019 (0.032, -0.029, 0.099) to add in queue.
[Open3D INFO] Picked point #7062 (-0.052, 0.0057, 0.098) to add in queue.


1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #849926 (-0.0071, -0.049, 0.56) to add in queue.
[Open3D INFO] Picked point #651047 (-0.019, 0.015, 0.65)

In [ ]:
# o3d.visualization.draw_geometries([pcd, obj_mesh_t])

In [36]:
frame = 557

mesh_hand = o3d.geometry.TriangleMesh()
mesh_hand.vertices = o3d.utility.Vector3dVector(hand_verts[frame])
mesh_hand.triangles = o3d.utility.Vector3iVector(hand_faces)

mesh_hand.compute_vertex_normals()
mesh_hand.paint_uniform_color([141 / 255, 184 / 255, 226 / 255])

pcd_joints = o3d.geometry.PointCloud()
pcd_joints.points = o3d.utility.Vector3dVector(hand_joints[frame])
pcd_joints.paint_uniform_color([73 / 255, 84 / 255, 94 / 255])

mesh_hand.scale(1/1000, center=(0,0,0))
pcd_joints.scale(1/1000, center=(0,0,0))

pts_palm = np.zeros((4,3))
pts_palm = hand_verts[frame][[229, 204, 183, 144],:]
pts_palm = hand_verts[frame][[204, 229, 144, 183],:]
pcd_palm = o3d.geometry.PointCloud()
pcd_palm.points = o3d.utility.Vector3dVector(pts_palm)
pcd_palm.paint_uniform_color([1,0,0])
pcd_palm.scale(1/1000, center=(0,0,0))

PointCloud with 4 points.

In [37]:
o3d.visualization.draw_geometries([mesh_hand, pcd_palm])

In [38]:
# AprilTag corners projected to the hand
corners_w = apriltags.corners_w(idx="000557")
points = corners_w[4:,:]
pos_mano = np.asarray(pcd_palm.points)


R, t, pos_mano_rt = rigid_transform_3D(pos_mano.T, points.T, transform_source=True)

In [39]:
pcd_joints_rt = copy.deepcopy(pcd_joints)
pcd_joints_rt.rotate(R, center=(0, 0, 0))
pcd_joints_rt.translate(t)

mesh_hand_rt = copy.deepcopy(mesh_hand)
mesh_hand_rt.rotate(R, center=(0, 0, 0))
mesh_hand_rt.translate(t)


TriangleMesh with 778 points and 1538 triangles.

In [41]:
o3d.visualization.draw_geometries([mesh_hand_rt, obj_mesh_t, pcd]) #, 

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
